In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Your team
* Joshua Finch
* Kyle Ormonde
* Ingrid Moe

# Task 1: Dataset
The first step is to find your own domain-specific dataset for your data mining project. The dataset should be complex enough so that it is not straightforward to find patterns with simple calculations (impossible without preprocessing and data mining approaches). There is no limit in size for the dataset, but typically a good sized data for mining is around 100k-100M. Minimum of 100k samples/rows and minimum of 100 attributes/columns. 

It could have thousands/millions rows (or columns or sometimes both rows/columns). A good data typically contains various types of data (numerical, nominal, ordinal, Boolean etc) with some errors (missing or dirty values etc) in the data. The dataset could be text data, tabular formatted data, georeferenced data. See possible data sources: Kaggle repository (https://www.kaggle.com/datasets).

In [23]:
# todo: import dataset(s) into pandas and print samples.
fpath = '/kaggle/input/us-accidents/US_Accidents_March23.csv'

#~~~ full data run ~~~
# df = pd.read_csv(fpath, header=0)

#~~~ no null lat/long ~~~
df = pd.read_csv(fpath, header=0, na_values=['nan']).dropna()

#~~~ random % of rows ~~~
df10 = df.sample(frac=0.1)

In [26]:
# Print sample rows
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
3402762,A-3412645,Source1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,...,False,False,False,False,False,False,Night,Night,Night,Night
3402767,A-3412650,Source1,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,...,False,False,False,False,False,False,Day,Day,Day,Day
3402771,A-3412654,Source1,2,2016-02-08 11:51:46,2016-02-08 17:51:46,41.375310,-81.820170,41.367860,-81.821740,0.521,...,False,False,False,False,False,False,Day,Day,Day,Day
3402773,A-3412656,Source1,2,2016-02-08 15:16:43,2016-02-08 21:16:43,40.109310,-82.968490,40.110780,-82.984000,0.826,...,False,False,False,False,False,False,Day,Day,Day,Day
3402774,A-3412657,Source1,2,2016-02-08 15:43:50,2016-02-08 21:43:50,39.192880,-84.477230,39.196150,-84.473350,0.307,...,False,False,False,False,False,False,Day,Day,Day,Day


In [101]:
from pandas.api.types import is_numeric_dtype, is_bool_dtype
# print(is_numeric_dtype(df10['Crossing']))
print(df10.dtypes)

ID                        object
Source                    object
Severity                   int64
Start_Time                object
End_Time                  object
Start_Lat                float64
Start_Lng                float64
End_Lat                  float64
End_Lng                  float64
Distance(mi)             float64
Description               object
Street                    object
City                      object
County                    object
State                     object
Zipcode                   object
Country                   object
Timezone                  object
Airport_Code              object
Weather_Timestamp         object
Temperature(F)           float64
Wind_Chill(F)            float64
Humidity(%)              float64
Pressure(in)             float64
Visibility(mi)           float64
Wind_Direction            object
Wind_Speed(mph)          float64
Precipitation(in)        float64
Weather_Condition         object
Amenity                     bool
Bump      

In [114]:
# data = df
data = df10

# for col in df.columns:
for col in data.columns:
    if is_numeric_dtype(data[col]) == False or is_bool_dtype(data[col]) == True:
        print(f"Column: {col}")
        print("-" * 30)  # Separator

        # Get the value counts and sort by count 
        value_counts = data[col].value_counts().sort_values(ascending=False)

        # Print the top 10 categories (you can adjust the number as needed)
        for category, count in value_counts.head(5).items():
            print(f"{category}: {count}")

        print("\n")  # Add a blank line between columns

Column: ID
------------------------------
A-4106711: 1
A-6023828: 1
A-3983361: 1
A-5132903: 1
A-7267133: 1


Column: Source
------------------------------
Source1: 355455


Column: Start_Time
------------------------------
2021-01-26 16:16:13: 27
2021-01-26 16:17:33: 18
2021-02-16 06:43:35: 15
2021-02-16 06:42:43: 14
2020-10-20 12:57:48: 10


Column: End_Time
------------------------------
2021-11-22 08:00:00: 9
2020-01-25 00:00:00: 7
2020-02-12 00:00:00: 6
2020-09-11 14:15:00: 6
2020-02-15 00:00:00: 6


Column: Description
------------------------------
A crash has occurred causing no to minimum delays. Use caution.: 976
Accident: 614
An unconfirmed report of a crash has been received. Use caution.: 357
A crash has occurred use caution.: 271
A crash has occurred with minimal delay to traffic. Prepare to slow or move over for worker safety.: 261


Column: Street
------------------------------
I-5 N: 3691
I-95 N: 3633
I-95 S: 3480
I-5 S: 3225
I-10 E: 2412


Column: City
----------------

In [83]:
# data = df
data = df10

for col in data.columns:
    if is_numeric_dtype(data[col]) == True:
        print(f"Column: {col}")
        print("-" * 30)  # Separator

        # Get the value counts and sort by count (descending)
        value_counts = data[col].value_counts().sort_values(ascending=False)

        # Print the top 10 categories (you can adjust the number as needed)
        for category, count in value_counts.head(5).items():
            print(f"{category}: {count}")

        print("\n")  # Add a blank line between columns

Column: Severity
------------------------------
2: 3348445
4: 112511
3: 68026
1: 25567


Column: Start_Lat
------------------------------
25.702455: 357
28.451602: 355
27.447751: 337
28.449924: 323
25.689146: 313


Column: Start_Lng
------------------------------
-80.332105: 358
-81.477678: 355
-81.479136: 323
-80.38287199999998: 313
-81.470923: 309


Column: End_Lat
------------------------------
28.450015000000004: 1039
25.701774: 851
25.684322: 794
28.449928000000003: 781
25.889378: 712


Column: End_Lng
------------------------------
-81.471375: 1037
-80.334179: 851
-80.41662099999998: 794
-81.47721899999998: 781
-80.293318: 699


Column: Distance(mi)
------------------------------
0.0: 320985
0.008: 12654
0.009: 12079
0.01: 11318
0.0069999999999999: 10786


Column: Temperature(F)
------------------------------
73.0: 82761
77.0: 78073
72.0: 77695
75.0: 76750
70.0: 75366


Column: Wind_Chill(F)
------------------------------
73.0: 82761
77.0: 78073
72.0: 77695
75.0: 76750
70.0: 7536

*Say something meaningful about your dataset. For example: Why did you select it? Is there anything interesting or special about it? What do or did hope to learn from it?*

# Data Set Overview
## What is it?
This dataset is a countrywide car accident dataset that covers 49 states of the USA from February 2016 to March 2023 containing 7.7 million records.

## Points of Interest:
The dataset contains a broad spectrum of information regarding all aspects of car accidents such as length of the road affected by the accident, duration, time, and severity. It also contains highly detailed weather data such as temperature, wind direction, wind speed, and precipitation. Additionally, the data set categorises different configurations of roads such as if they contain a give-way, a railway, roundabouts, junctions, etc.

## Learning Goals:
The owner of this dataset has hopes that it can be used for a vast amount applications such as real-time accident prediction, studying accident hotspot locations, casualty analysis and potentially even identifying cause and effect rules to predict accidents.(Sobhan Moosavi, January 2021)
We hope to be able to identify which combination of driving conditions, outside of the obvious ones, are more likely to result in an accident in order to know what to be more aware of.

# Task 2: business scenarios
*List one or more possible questions you would like to investigate using your dataset. You may start this project with one set of questions but (after exploring the dataset) finish with a new set of questions and answers.*

## Accident Analysis

### Time
1. **Annual**: When is the most and least likely times for an accident to occur per year?
2. **Monthly**: When is the most and least likely times for an accident to occur per month?
3. **Daily**: When is the most and least likely times for an accident to occur per 24 hours?

### Location
1. **County**: Where is the most likely place for accidents to occur per county?
2. **State**: Where is the most likely place for accidents to occur per state?
3. **City**: Where is the most likely place for accidents to occur per city?

### Environmental Conditions
1. **Weather**: What are the most and least common crash causing environmental conditions (precipitation, wind)?
2. **Conditions**: What are the most and least common crash causing driving conditions (visibility)?

### Infrastructure
1. What are the most and least common crash causing road infrastructures (roundabouts, traffic lights, etc)?

In [ ]:
# some python code and results to support your business cases
 df.describe(include='all')  # see prac-1. 


# NOTE! This is a data-based project. Make sure your comments are based on printed code outputs and/or graphs.

# Task-3: preprocessing
See prac-2. Apply one or more preprocessing techniques

In [ ]:
# TODO: python code with outputs
# Add some comments to explain what and why you did.
# NOTE! unless you have a very good reason, do not drop rows nor columns
data = df.copy()

print('The dataset has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
print(data.describe())

In [ ]:
df.dtypes.value_counts()



In [ ]:

df['ID'] = pd.to_numeric(df['ID'], errors='coerce') #convert Long attribute to numeric


print(df.dtypes.value_counts()) #HELP

In [ ]:
numeric_attributes = df.columns[df.dtypes != "object"]
categorical_attributes = df.columns[df.dtypes == "object"]

print(numeric_attributes)
print(categorical_attributes) # what am i doing wrong? 

# Task-4: ARM

In [ ]:
# Apply one or more ARM techiques, see prac-3-ARM
# Report your results based on your code output

In [192]:
# Removed astro and naut twilights for testing

apri_data = df10[['Severity','State','Wind_Direction','Weather_Condition','Sunrise_Sunset','Civil_Twilight','Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal']]
apri_data.head()

,Severity,State,Wind_Direction,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal
4076053,2,CA,S,Fair,False,False,False,False,False,False,False,False,False,False,False,False
3647311,2,CA,ENE,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False
6512805,2,PA,NW,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True
6039315,2,PA,CALM,Fair,False,False,False,False,False,False,False,False,False,False,False,True
3953681,2,UT,NNW,Fair,False,False,False,False,False,False,False,False,False,False,False,False


In [193]:
apri_data['Severity'] = apri_data['Severity'].astype(str)

items = set()
for col in apri_data:
    items.update(apri_data[col].unique())
    print(f"{col}: {apri_data[col].unique()}")

/tmp/ipykernel_33/4226448066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apri_data['Severity'] = apri_data['Severity'].astype(str)


Severity: ['2' '1' '4' '3']
State: ['CA' 'PA' 'UT' 'NY' 'CT' 'MN' 'DC' 'FL' 'VA' 'AZ' 'NC' 'LA' 'OH' 'TX'
 'MD' 'MI' 'OR' 'NJ' 'SC' 'TN' 'GA' 'MT' 'KS' 'CO' 'ID' 'WA' 'NE' 'IN'
 'OK' 'AR' 'IA' 'MA' 'MO' 'AL' 'IL' 'WV' 'MS' 'WI' 'NH' 'NV' 'KY' 'DE'
 'NM' 'RI' 'ND' 'WY' 'ME' 'VT' 'SD']
Wind_Direction: ['S' 'ENE' 'NW' 'CALM' 'NNW' 'W' 'NNE' 'SSE' 'E' 'SSW' 'SE' 'WSW' 'VAR'
 'N' 'NE' 'SW' 'WNW' 'ESE' 'North' 'East' 'South' 'West' 'Variable']
Weather_Condition: ['Fair' 'Light Rain' 'Mostly Cloudy' 'Cloudy' 'Partly Cloudy' 'Light Snow'
 'Blowing Snow / Windy' 'Rain' 'Smoke' 'Haze' 'Cloudy / Windy'
 'Shallow Fog' 'Light Drizzle' 'Fog' 'Mostly Cloudy / Windy'
 'Blowing Snow' 'Heavy Rain' 'Fair / Windy' 'Thunder in the Vicinity'
 'Thunder' 'Partly Cloudy / Windy' 'Showers in the Vicinity'
 'Heavy T-Storm' 'Light Snow / Windy' 'Heavy Snow'
 'Light Rain with Thunder' 'Snow' 'T-Storm' 'Snow and Sleet' 'Overcast'
 'Wintry Mix' 'N/A Precipitation' 'Heavy Snow / Windy' 'Haze / Windy'
 'Drizzle and Fo

In [195]:
# Run this or the one below only. Delete one later.

itemset = set(items)
encoded_vals = []
for index, row in apri_data.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]
ohe_df = pd.DataFrame(encoded_vals)
ohe_df.head()

,True,Sleet / Windy,IA,Light Snow with Thunder,SE,Thunder,Cloudy,Light Ice Pellets,WI,T-Storm,...,RI,Snow and Sleet,AL,Cloudy / Windy,SSW,False,CA,S,2,Fair
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1


In [196]:
# Similar to above, but instead categorises the data into boolean values, one column for each unique value. The above version doesn't handle true/false column headers well.
apri_data = pd.get_dummies(apri_data)

itemset = set(apri_data.columns)
encoded_vals = []

for index, row in apri_data.iterrows():
    labels = {}
    for item in itemset:
        labels[item] = row[item]
    encoded_vals.append(labels)

ohe_df = pd.DataFrame(encoded_vals)

In [198]:
freq_items = apriori(ohe_df, min_support=0.2, use_colnames=True, verbose=1)
freq_items.sort_values(by='support', ascending=False)

Processing 6 combinations | Sampling itemset size 2


,support,itemsets
2,0.942294,(Severity_2)
1,0.477641,(Weather_Condition_Fair)
4,0.453320,"(Severity_2, Weather_Condition_Fair)"
0,0.247924,(State_CA)
3,0.243184,"(State_CA, Severity_2)"


In [187]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.5)
rules.sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


# Task-5: Classification

In [ ]:
# apply one or more classification methods, see pracs-4 and 5
# Report your results based on your code output

# Task-6: Clustering

In [ ]:
# apply one or more clustering methods, see pracs-6 and 7
# Report your results based on your code output

# Task-7: Project specific and final results

In [ ]:
# The preceding tasks may or may not produce interesting results. They are mandatory exploration tasks.
# In this section you need to focus on anything interesting you found so far, which is specific to your data. 
# Some additional code and output may be needed here to make any final conclusions.